In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *

In [3]:
path= Path('PCImages')

In [4]:
path2fm= 'FeatureMatsMerged/TheGreatCollection.txt'
path2colnames= 'FeatureMatIndex.txt'

In [5]:
fm= fm_from_txt(path2fm, path2colnames)
fm.shape

(56657, 105)

In [14]:
#export
class ToFloatTensor(object): 
    def __call__(self, img): return torch.from_numpy(np.array(img, dtype=np.float32, copy=False))

In [246]:
ds= CellDataSet(path, fm, label = 'label1', cell_phase = None, trfms= [CenterCrop(), ToFloatTensor()], 
               random_sample=100)

In [247]:
len(ds)

100

In [285]:
r,c=ds[0][0].shape
r

256

In [340]:
bs= 4
train_dl, valid_dl= ds2dls(ds, bs) 

## Model

In [341]:
# Device configuration
device = torch.device('cuda')

In [342]:
#export
import torchvision.models as models
from torch import nn

In [391]:
#export
class Resize(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f

    def forward(self, x): return self.f(x)

In [392]:
#export
def cell_resize(x, size): return x.view(-1, 1, size, size)

In [398]:
#export
def get_model(m, c = 7):
    m.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
    m.fc= nn.Linear(in_features=512, out_features=c).cuda()
    return nn.Sequential(
        Resize(cell_resize),
        m
    )

In [399]:
model = get_model(models.resnet34(pretrained=True).cuda())

In [376]:
#model.eval()

In [377]:
loss_func= nn.CrossEntropyLoss()

In [378]:
lr= 1e-5
opt= torch.optim.Adam(model.parameters(), lr=lr)

In [379]:
#export
def accuracy(outb, yb): return (torch.argmax(outb, dim=1)==yb).float().mean()

In [380]:
metric= accuracy

In [ ]:
#export
class CellLearner():
    def __init__(self, model, loss_function, optimizer, dataset):
        self.model= get_model(model.cuda())

## Training loop

In [381]:
epochs = 1

In [382]:
device = torch.device('cuda',0)

In [394]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl:
            xb= xb.to(device)
            yb= yb.to(device)
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                xb= xb.to(device)
                yb= yb.to(device)
                pred = model(xb)
                pred= pred.to(device)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
                nv = len(valid_dl)
    print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [395]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

TypeError: cell_resize() missing 1 required positional argument: 'size'